In [ ]:
import os
import numpy as np
import dlib 
import cv2
from numpy import save
from PIL import Image

In [ ]:
#5 frames
data_dir = "./Female"
img_height , img_width = 64, 64
seq_len = 5
 
YawDD = os.listdir("./Female")
classes = []
for item in YawDD:
    if item == ".DS_Store" or item == ".ipynb_checkpoints":
        pass
    else:
        classes.append(item)
classes = classes[0:] 
classes

In [ ]:
def detect_faces(image):

    face_detector = dlib.get_frontal_face_detector()

    # Run detector and get bounding boxes of the faces on image.
    detected_faces = face_detector(image, 1)
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]

    return face_frames

In [ ]:
def frames_extraction_gray(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            detected_faces = detect_faces(img_gray)
            
            for face_rect in detected_faces:
                face = Image.fromarray(img_gray).crop(face_rect)
                face = np.array(face).astype(np.uint8)
                image = cv2.resize(face, (img_height, img_width), interpolation=cv2.INTER_LINEAR)

                frames_list.append(image)
                count += 1
        else:
            print("Defected frame")
            break

    return frames_list

In [ ]:
def create_data_gray(input_dir):
    X = []
    Y = []
          
    for c in classes:
        
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames = frames_extraction_gray(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                X.append(frames)
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [ ]:
X_gray, Y_gray = create_data_gray(data_dir)

In [ ]:
save('input_Female_gray5.npy', X_gray)
save('output_Female_gray5.npy', Y_gray)